In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Assignment

For this lab, we're going to use everything we've learned so far and try to predict bike demand using linear models!

The data comes from this [page](https://www.kaggle.com/datasets/lakshmi25npathi/bike-sharing-dataset).

    1. Download the dataset. 
    
In fact, by doing so you'll get two datasets. One is *daily* bike demand, and the other is *hourly* bike demand.
The daily dataset has fewer records (731) and might be a bit simpler to work with if you're just getting started.
If you want to have a bit more fun, go for the hourly dataset (17379 records).

Pick the one you want! 

    2. Read carefully through the documentation for the dataset. 

The download will also include a *readme* with good information about the dataset and descriptions for all the columns in it. The target we're going to try to model is the column *cnt*. 

You are forbidden to use the columns *casual* and *registered*. Why do you think that is?

    3. Conduct some initial EDA.

Get to know the data by e.g. doing some initial plots and some statistics in order to try to understand what you're dealing with. 

While doing this, also think carefully about what features you'd think are relevant to what we're trying to predict. 

Which feature(s) do you have reasons to believe might be the individually most reliable one(s) to predict the demand?

    4. Clean data

If warranted, start cleaning data.

    5. Model

Pick the features you think or have reasons to believe are relevant. Then, train a linear model and evaluate it. Does it perform well?

In this step, you are of course allowed to train many different models (all with different features). 

Can you find a particular combination of features that seem to work best?



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# load the dataset
bike_rental_df = pd.read_csv('../../data/bike_rental/hour.csv')


In [ ]:
bike_rental_df.info()

In [ ]:
bike_rental_df.describe()

In [ ]:
# The number of unique values per column
print("Number of unique values per column:\n")
for col in bike_rental_df.columns:
    if len(col)<6:
        sep = "\t\t"
    else:
        sep = "\t"
    print(f'{col}: {sep}{len(bike_rental_df[col].unique())}')

In [ ]:
bike_rental_df.head()


In [ ]:
# Remove instant, dteday, casual and registered columns.

columns_to_keep = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'cnt']

bike_rental_reduced_df = bike_rental_df[columns_to_keep].copy()
bike_rental_reduced_df.head()

In [ ]:
# Check the distribution of the cnt column, our targets. Anything that looks odd?

plt.hist(bike_rental_reduced_df['cnt'], bins=50)
plt.show()


So a lot of hours have very few rentals. That makes sense, and would probably make up a majority of the nighttime hours. Nothing too odd here, I think.

I'll go on and plot the other columns. The continuous ones as scatterplots and the categorical ones as countplots just to see how the number of records vary. Then I'll put boxplots next to both kinds of plots to see the distribution (even if this might make less sense for the continuous features).

In [ ]:
# Define functions for the two types of plotting.

def plot_features_against_cnt_continuous(feature):
    print(f"________________________________________ {feature} ________________________________________")
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)

    X, y = bike_rental_reduced_df[feature], bike_rental_reduced_df["cnt"]
    plt.scatter(X, y)
    plt.xlabel(feature)
    plt.ylabel('Number of rentals')

    plt.subplot(1, 2, 2)
    sns.boxplot(data=bike_rental_reduced_df, x=feature, y="cnt", color="lightblue")
    plt.xlabel(feature)
    plt.ylabel('Value distribution, number of rentals')

    plt.tight_layout()
    plt.show()

def plot_features_against_cnt_categories(feature):
    print(f"________________________________________ {feature} ________________________________________")
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    sns.countplot(data=bike_rental_reduced_df, x=feature)
    plt.xlabel(feature)
    plt.ylabel('Number of records')

    plt.subplot(1, 2, 2)
    sns.boxplot(data=bike_rental_reduced_df, x=feature, y="cnt", color="lightblue")
    plt.xlabel(feature)
    plt.ylabel('Value distribution, number of rentals')

    plt.tight_layout()
    plt.show()


In [ ]:
# Apply the plotting functions to the features.

continuous_cols = ['temp', 'atemp', 'hum', 'windspeed']
for col in continuous_cols:
    plot_features_against_cnt_continuous(col)

categorical_cols = ['yr', 'mnth', 'hr', 'season', 'holiday', 'weekday', 'workingday', 'weathersit']
for col in categorical_cols:
    plot_features_against_cnt_categories(col)

From this, even if there's a great spread in the results, atleast we can see that there is a general trend that higher temp and atemp means more rentals, though atleast for temp it seems to dip a bit at very high temperatures (could be 2nd order). There's a declining trend in rentals as it gets more humid and more windy. 

There are seasonal differences, with more rentals in the summer months (and late spring/early autumn) and the number of rentals seem to have increased from year 2011 to year 2012.

There are distinct peaks for commute hours. Naturally, nighttime sees fewer rentals. 

Weather seems to be a good indicator. Not sure whether to treat weathersit as nominal or ordinal. It gets worse as the numbers increase. But are they comparable. I'll treat them as nominal for now.

Now, can and should we keep all these features? If so we need to do one-hot encoding for the categorical data that is not already binary.



The columns that seem to be most relevant for predicting the number of rentals are:
* temp
* atemp
* humidity
* windspeed
* hr
* season 
* workingday
* holiday
* weathersit
* yr

I decided to keep workingday, though it is really just a combination of weekday and holiday, and remove weekday. This is to reduce the number of dimensions. Though that means that holiday is represented in two of the features. Not sure if that matters.

I also decided to remove month and keep season in order to reduce the number of dimensions.

Out of these, the first four are continuous. 

Holiday, workingday and yr are actually binary, already with either 1 or 0 as values. So these we can keep as is. 

Season and weathersit have categories looking like numbers. We need to do one-hot encoding for these.

What about hr? Does it make sense to treat hour as continuous? Then how do we compare hour 23 to hour 0? So ... no.

Also 24 columns will not do. I think it would be better to categorize hr into "time of day". Do these need to be of equal size (representing equal numbers of hours)? Not sure. Looking at the data, it would make more sense to have them as "night", "commute hours", "daytime", "evening". But they would not be of equal number of hours. I'll go ahead and try it out.

In [ ]:
# Not sure about this. The other way to do it would be to divide 24 hours into equal chunks of time: 
# night, morning, day, evening.

def get_time_of_day(hour):
    if hour < 7:
        time_of_day = "night";
    elif hour < 9:
        time_of_day = "commute"
    elif hour < 17:
        time_of_day = "daytime"
    elif hour < 20:
        time_of_day = "commute"
    else:
        time_of_day = "evening"
    return time_of_day



In [ ]:
bike_rental_reduced_df.loc[:, "time_of_day"] = bike_rental_reduced_df['hr'].apply(get_time_of_day)

bike_rental_reduced_df.head()

In [ ]:
# Check the plots for the new time of day column.
plot_features_against_cnt_categories("time_of_day")

In [ ]:
final_columns_to_keep = ['yr', 'holiday', 'workingday', 'temp', 'atemp',
       'hum', 'windspeed', 'season', 'weathersit', 'time_of_day', 'cnt']

bike_rental_twice_reduced_df = bike_rental_reduced_df[final_columns_to_keep].copy()

bike_rental_twice_reduced_df.head()


In [ ]:
# Do one hot encoding on season, weathersit and time_of_day
bike_rental_onehot_df = pd.get_dummies(bike_rental_twice_reduced_df, columns=['season', 'weathersit', 'time_of_day'], dtype=int, drop_first=True)

bike_rental_onehot_df.head()

In [ ]:
X, y = bike_rental_onehot_df.drop(columns=['cnt']).values, bike_rental_onehot_df['cnt'].values

y = y.reshape(-1,1)

print(X.shape)
print(y.shape)

In [ ]:
bike_rental_onehot_df.columns

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # set a random state, so we can reproduce our results

print('Train set:')
print('X:', len(X_train))
print('y:', len(y_train), end='\n\n')

print('Test set:')
print('X:', len(X_test))
print('y:', len(y_test))

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression() 
lr.fit(X_train, y_train)

In [ ]:
# print the weights of the trained model

print(lr.intercept_)
print(lr.coef_)

In [ ]:
# predictions for both the train and test set

y_train_hat = lr.predict(X_train)
y_test_hat = lr.predict(X_test)

In [ ]:
# calculate MSE for both sets

print('Train:')
print(f'MSE: {mean_squared_error(y_train, y_train_hat)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_train, y_train_hat))}')

print('Test:')
print(f'MSE: {mean_squared_error(y_test, y_test_hat)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_test_hat))}')

____

A little bit of error analysis. Is the above result good or bad?

In [ ]:
# we can plot our predictions against the actual values

plt.plot(y_test, y_test_hat, 'o');
plt.xlabel('Actual'); 
plt.ylabel('Predicted'); 
plt.title('Actual vs Predicted');


Apart from looking at the above plot to understand strengths and weaknesses of your model (we can see that our model seem to perform poorly on actual valeus that are either very low, or very high) we can also create stratas of our test_set and plot only the results for them.

____

Whoa. That's huge. I guess it's positive that it's actually lower for the test data. But let's go back and adjust.


In [ ]:
# Listing possible feature combinations to try

# Focus on date and time features
features_try_1 = ['holiday', 'workingday', 'season_2', 'season_3', 'season_4', 'time_of_day_daytime',
       'time_of_day_evening', 'time_of_day_night']

# Focus on weather features
features_try_2 = ['temp', 'atemp', 'hum', 'windspeed', 'weathersit_2',
       'weathersit_3', 'weathersit_4']

# Focus on my own idea of what should matter the most (okay most features are kept here...)
features_try_3 = ['holiday', 'workingday', 'temp', 'hum', 'windspeed',
       'season_2', 'season_3', 'season_4', 'weathersit_2',
       'weathersit_3', 'weathersit_4', 'time_of_day_daytime',
       'time_of_day_evening', 'time_of_day_night']

# Try another balance, perhaps we're including things like humidity and windspeed twice when 
# applying them along with weathersit?
features_try_4 = ['yr', 'workingday', 'temp', 'hum', 'windspeed', 'time_of_day_daytime',
       'time_of_day_evening', 'time_of_day_night']

# The other way around, excluding hum and windspeed but keeping weathersit
features_try_5 = ['yr', 'workingday', 'season_2', 'season_3', 'season_4', 'weathersit_2',
       'weathersit_3', 'weathersit_4', 'time_of_day_daytime',
       'time_of_day_evening', 'time_of_day_night']

for featureset in [features_try_1, features_try_2, features_try_3, features_try_4, features_try_5]:
    
    print(f"Trying: {featureset}\n")

    X, y = bike_rental_onehot_df[featureset].values, bike_rental_onehot_df['cnt'].values

    y = y.reshape(-1,1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # set a random state, so we can reproduce our results

    lr = LinearRegression()
    lr.fit(X_train, y_train)

    print(f"Intercept: {lr.intercept_}")
    print(f"Coef: {lr.coef_}")

    y_train_hat = lr.predict(X_train)
    y_test_hat = lr.predict(X_test)

    print('\nTrain:')
    print(f'MSE: {mean_squared_error(y_train, y_train_hat)}')
    print(f'RMSE: {np.sqrt(mean_squared_error(y_train, y_train_hat))}')
    
    print('\nTest:')
    print(f'MSE: {mean_squared_error(y_test, y_test_hat)}')
    print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_test_hat))}\n')


So all my attempts to reduce the number of features of this model resulted in worse results. 

I guess it's just not possible to get a better fit on a linear model, or I haven't found the optimal set of features. 